
# Pytorch를 활용한 미국드라마 프렌즈 대사 감정 분석 모델(Onehot)
## 모델 : ELECTRA Base <br>

## Dataset
- EmotionLines 제공 프렌즈 대사 데이터셋<br>
- 기존 8개 감정 분류를 긍정, 부정의 0,1로 재분류 실시<br>

## References
- https://github.com/jiwonny/nlp_emotion_classification/blob/master/friends_electra.ipynb (소스코드 참조)
- http://doraemon.iis.sinica.edu.tw/emotionlines/index.html
- https://huggingface.co/transformers/training.html
- http://wikidocs.net/book/2155 (데이터 구조 분석 부분 참조)

## 개발 환경
  - Google Corab (With GPU)<br>
  - 구글 드라이브 연동 후 본인 경로 설정 필수<br>

In [ ]:
#구글 코랩 환경에서 필수 다운로드 요소인 'Transformers' 불러오기
!pip install transformers

     |████████████████████████████████| 1.5MB 13.1MB/s 
     |████████████████████████████████| 890kB 46.8MB/s 
     |████████████████████████████████| 2.9MB 50.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=bef355f0224aad3475417a029f4d22a419602ce99ea2da54da4120717d14b309
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# 구글 드라이브 마운트하기

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 모델 및 데이터 파일 다운로드
!git clone https://github.com/smartsms85/NLP_Sentiment_Analysis.git

In [ ]:
# 모델 생성에 필요한 각종 필수 도구 Import

import torch

from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import random
import time
import datetime
import json

In [ ]:
# 판다스를 활용한 Json 데이터 프레임 생성 함수 설정

def jsonToDf(file_name):
  with open(file_name, encoding = 'utf-8', mode = 'r') as file:
    json_array = json.load(file)
  result = pd.DataFrame.from_dict(json_array[0])
  is_first = True
  for array in json_array:
      
    if is_first:
        is_first = False
        continue
    
    temp_df = pd.DataFrame.from_dict(array)
    result = result.append(temp_df, ignore_index = True)

  return result

In [ ]:
# Json 데이터 불러오기  (Default : 깃 클론 지정 폴더 / 필요시 본인 경로 설정 필요)
train = jsonToDf('/content/NLP_Sentiment_Analysis/FRIENDS/json/friends_train.json')
dev = jsonToDf('/content/NLP_Sentiment_Analysis/FRIENDS/json/friends_dev.json')
test = jsonToDf('/content/NLP_Sentiment_Analysis/FRIENDS/json/friends_test.json')

In [ ]:
# train 데이터셋 중 emotion 라벨 재할당
train['emotion'].replace('surprise', 1, inplace = True)
train['emotion'].replace('joy', 1, inplace = True)
train['emotion'].replace('anger', 0, inplace = True)
train['emotion'].replace('disgust', 0, inplace = True)
train['emotion'].replace('fear', 0, inplace = True)
train['emotion'].replace('sadness', 0, inplace = True)
train['emotion'].replace('non-neutral', 0, inplace = True)
train['emotion'].replace('neutral', np.nan , inplace = True)
train = train.dropna(how = 'any')
print(train.groupby('emotion').size().reset_index(name = 'count'))

   emotion  count
0      0.0   3306
1      1.0   2503


In [ ]:
# dev 데이터셋 중 emotion 라벨 재할당
dev['emotion'].replace('surprise', 1, inplace = True)
dev['emotion'].replace('joy', 1, inplace = True)
dev['emotion'].replace('anger', 0, inplace = True)
dev['emotion'].replace('disgust', 0, inplace = True)
dev['emotion'].replace('fear', 0, inplace = True)
dev['emotion'].replace('sadness', 0, inplace = True)
dev['emotion'].replace('non-neutral', 0, inplace = True)
dev['emotion'].replace('neutral', np.nan , inplace = True)
dev = dev.dropna(how = 'any')
print(dev.groupby('emotion').size().reset_index(name = 'count'))

   emotion  count
0      0.0    413
1      1.0    274


In [ ]:
# test 데이터셋 중 emotion 라벨 재할당
test['emotion'].replace('surprise', 1, inplace = True)
test['emotion'].replace('joy', 1, inplace = True)
test['emotion'].replace('anger', 0, inplace = True)
test['emotion'].replace('disgust', 0, inplace = True)
test['emotion'].replace('fear', 0, inplace = True)
test['emotion'].replace('sadness', 0, inplace = True)
test['emotion'].replace('non-neutral', 0, inplace = True)
test['emotion'].replace('neutral', np.nan , inplace = True)
test = test.dropna(how = 'any')
print(test.groupby('emotion').size().reset_index(name = 'count'))

   emotion  count
0      0.0    887
1      1.0    590


# Train, Dev, Test, Predict 데이터 전처리 과정

In [ ]:
# Train,Dev,Test 데이터의 전처리 진행을 위한 함수 설정
# Max Len은 전체 데이터셋의 95%를 수준을 커버하는 100으로 설정

MAX_LEN = 100

def getInputsAndLabels(dataset):
  data = dataset.copy(deep=True)
  data['utterance'] = data['utterance'].str.lower()

  utterances = data['utterance']
  utterances = ["[CLS] " + str(utterance) + " [SEP]" for utterance in utterances]
  
  encoder = LabelEncoder()
  labels = data['emotion'].values
  encoder.fit(labels)
  labels = encoder.transform(labels)

  tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, labels, attention_masks

In [ ]:

# Predict Data 전처리를 위한 함수 설정 

def getInputsFromTest(dataset):
  data = dataset.copy(deep=True)
  data['utterance'] = data['utterance'].str.lower()

  utterances = data['utterance']
  utterances = ["[CLS] " + str(utterance) + " [SEP]" for utterance in utterances]
  
  tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, attention_masks

In [ ]:
def getIndex(dataset):
  data = dataset.copy(deep = True)
  input_index = data.id.tolist()
  return torch.tensor(input_index)

In [ ]:
# Train, Dev, Test(생략) 및 Predict 각 각의 데이터에 맞는 전처리 함수 적용
train_inputs, train_labels, train_masks = getInputsAndLabels(train)
dev_inputs, dev_labels, dev_masks = getInputsAndLabels(dev)


In [ ]:
# 파이토치 텐세로 변환

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

dev_inputs = torch.tensor(dev_inputs)
dev_labels = torch.tensor(dev_labels)
dev_masks = torch.tensor(dev_masks)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy 

In [ ]:
# 파이토치의 DataLoader로 입력
# 학습시 배치 사이즈 만큼 데이터를 가져옴

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels)
dev_sampler = SequentialSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=2)
model.cuda()

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.d

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

# Train & Validation

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8
                )
# 5 -> 15 -> 5
epochs = 5

total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


In [ ]:
from sklearn.metrics import f1_score

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def getF1Score(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  return f1_score(labels_flat, pred_flat, average = None)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device).long() for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
             
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()


        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy, eval_f1 = 0, 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in dev_dataloader:
        batch = tuple(t.to(device).long() for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
     
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        # tmp_eval_f1 = getF1Score(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        # eval_f1 += tmp_eval_f1
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    # print("  F1: {0:.2f}".format(eval_f1/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...

  Average training loss: 0.58
  Training epcoh took: 0:00:36

Running Validation...
  Accuracy: 0.76
  Validation took: 0:00:01

======== Epoch 2 / 5 ========
Training...

  Average training loss: 0.47
  Training epcoh took: 0:00:36

Running Validation...
  Accuracy: 0.75
  Validation took: 0:00:01

======== Epoch 3 / 5 ========
Training...

  Average training loss: 0.38
  Training epcoh took: 0:00:36

Running Validation...
  Accuracy: 0.77
  Validation took: 0:00:01

======== Epoch 4 / 5 ========
Training...

  Average training loss: 0.32
  Training epcoh took: 0:00:36

Running Validation...
  Accuracy: 0.76
  Validation took: 0:00:01

======== Epoch 5 / 5 ========
Training...

  Average training loss: 0.27
  Training epcoh took: 0:00:36

Running Validation...
  Accuracy: 0.76
  Validation took: 0:00:01

Training complete!
